In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd # data wrangling
# matplotlib, seaborn # data visualization
# sklearn -> ML
# Tensorflow, Keras -> DL
# Stats

In [ ]:
adv = pd.read_csv('/kaggle/input/trainings/Advertising.csv')
adv.head()

In [ ]:
insurance = pd.read_csv('/kaggle/input/trainings/insurance.csv')
insurance.head()

In [ ]:
type(insurance)

In [ ]:
insurance.shape

In [ ]:
# Tuple -> (5, 6, 9, 10)
# List -> [10, 30, 49]

In [ ]:
# supervised learning models -> past data (labelled data)
# Regression analysis

### Regression analysis
- Linear regression
- Tree based models - Tree regressors
- SVR
- ...
- ...
- Neural networks

### Classification analysis
- Logistic regression
- Tree based models - Tree classifiers
- SVM
- Naive bayes
- Neural networks

In [ ]:
target_col = 'expenses'


#### Pre processing - Linear Regression
- Missing values treatment
- Outlier treatment
- Convert all inputs to numeric
- Standardize
- Transformation (PCA)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# individual cols in dataframe are series
insurance.isna().sum()

In [ ]:
insurance['expenses'].plot.box()

In [ ]:
adult = pd.read_csv('/kaggle/input/trainings/adult.csv')
adult.shape

In [ ]:
adult.head()

In [ ]:
adult['age'].plot.box()

In [ ]:
pd.get_dummies(insurance[['smoker', 'age']], drop_first=True).head(2)

In [ ]:
insurance_dummies = pd.get_dummies(insurance, drop_first=True)
insurance_dummies.head()

In [ ]:
input_data = insurance_dummies.drop(['expenses'], axis=1)
scaler = StandardScaler().fit(input_data)
std_input_data = scaler.transform(input_data)
df_std_input_data = pd.DataFrame(std_input_data, columns=input_data.columns)
df_std_input_data.head()

In [ ]:
target_col = 'expenses'
input_cols = insurance_dummies.columns.drop(['sex_male', 'region_northwest', 'region_southeast',
                                    'region_southwest', target_col])

train_x, test_x, train_y, test_y = train_test_split(insurance_dummies[input_cols],
                                                   insurance_dummies[target_col],
                                                   test_size=0.2,
                                                   random_state=1)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

In [ ]:
scaler = StandardScaler().fit(train_x)
train_x_scaled = scaler.transform(train_x)
test_x_scaled = scaler.transform(test_x)
df_train_x_scaled = pd.DataFrame(train_x_scaled, columns=train_x.columns, index=train_x.index)
df_test_x_scaled = pd.DataFrame(test_x_scaled, columns=test_x.columns, index=test_x.index)

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(data=insurance, x='bmi', y='expenses')

In [ ]:
model = LinearRegression().fit(df_train_x_scaled, train_y)
model

In [ ]:
df_model_coef = pd.DataFrame({
    'variable': df_train_x_scaled.columns,
    'coef': model.coef_
})
df_model_coef

In [ ]:
model.intercept_

In [ ]:
test_y_pred = model.predict(df_test_x_scaled)
df_error = pd.DataFrame({
    'actual': test_y,
    'predicted': test_y_pred
})
df_error['error'] = df_error['actual'] - df_error['predicted']
df_error['square_error'] = np.square(df_error['error'])
df_error['square_error'].sum() / df_error.shape[0]

In [ ]:
mean_squared_error(test_y, test_y_pred)

In [ ]:
rmse = np.sqrt(mean_squared_error(test_y, test_y_pred))
rmse

In [ ]:
r2_score(test_y, test_y_pred)

In [ ]:
sns.pairplot(insurance, hue='smoker')

In [ ]:
insurance_dummies['smoker_yes'].head()

In [ ]:
target_col = 'expenses'

insurance_dummies = pd.get_dummies(insurance, drop_first=True)
insurance_dummies['bmi_high'] = insurance_dummies['bmi'].apply(lambda v: 1 if v > 29 else 0)
insurance_dummies['bmi_high_smoker'] = insurance_dummies['bmi_high'] * insurance_dummies['smoker_yes'] 
#insurance_dummies['age_smoker'] = insurance_dummies['age'] * insurance_dummies['smoker_yes']
#insurance_dummies['bmi_square'] = np.square(insurance_dummies['bmi'])
input_cols = insurance_dummies.columns.drop(['sex_male', 'region_northwest', 'region_southeast',
                                    'region_southwest', 'bmi', 'bmi_high', target_col])

train_x, test_x, train_y, test_y = train_test_split(insurance_dummies[input_cols],
                                                   insurance_dummies[target_col],
                                                   test_size=0.2,
                                                   random_state=1)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

scaler = StandardScaler().fit(train_x)
train_x_scaled = scaler.transform(train_x)
test_x_scaled = scaler.transform(test_x)
df_train_x_scaled = pd.DataFrame(train_x_scaled, columns=train_x.columns, index=train_x.index)
df_test_x_scaled = pd.DataFrame(test_x_scaled, columns=test_x.columns, index=test_x.index)

model = LinearRegression().fit(df_train_x_scaled, train_y)
test_y_pred = model.predict(df_test_x_scaled)
print(r2_score(test_y, test_y_pred))

In [ ]:
# numerical vs numerical: correlation
# categorical vs numerical: t-test / ANNOVA
# categorical vs categorical: chi-square test

In [ ]:
insurance_dummies.corr()

In [ ]:
import statsmodels.api as sm
df_train_x_scaled_subset = df_train_x_scaled
data_with_intercept = sm.add_constant(df_train_x_scaled_subset)
model = sm.OLS(train_y, data_with_intercept)
res = model.fit()
res.summary()

# Logistic Regression

In [ ]:
adult['income'].value_counts().plot.pie()

In [ ]:
adult['income'].value_counts() / adult.shape[0] * 100

In [ ]:
adult['predicted'] = '<=50K'
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(adult['income'], adult['predicted']))
print(f1_score(adult['income'], adult['predicted'], pos_label='>50K'))

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-1 * x))

In [ ]:
y_hat = np.linspace(-100,100,10000)
y_transformed = []
for i in y_hat:
    y_transformed.append(sigmoid(i))
import matplotlib.pyplot as plt

plt.plot(y_hat, y_transformed)

In [ ]:
adult['income'].unique()

In [ ]:
adult = pd.read_csv('/kaggle/input/trainings/adult.csv')
#adult['income'] = adult['income'].replace({'<=50K': 0, '>50K': 1})
#target_col = 'income'

#adult_dummies = pd.get_dummies(adult.drop('income', axis=1), drop_first=True)
#adult_dummies['income'] = adult['income']
#adult_dummies.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
adult = pd.read_csv('/kaggle/input/trainings/adult.csv')
adult['income'] = adult['income'].replace({'<=50K': 0, '>50K': 1})


adult_dummies = pd.get_dummies(adult, drop_first=True)
target_col = 'income'
input_cols = adult_dummies.columns.drop(target_col)

train_x, test_x, train_y, test_y = train_test_split(adult_dummies[input_cols],
                                                   adult_dummies[target_col],
                                                   test_size=0.2,
                                                   random_state=1)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

scaler = StandardScaler().fit(train_x)
train_x_scaled = scaler.transform(train_x)
test_x_scaled = scaler.transform(test_x)
df_train_x_scaled = pd.DataFrame(train_x_scaled, columns=train_x.columns, index=train_x.index)
df_test_x_scaled = pd.DataFrame(test_x_scaled, columns=test_x.columns, index=test_x.index)


model = LogisticRegression().fit(df_train_x_scaled, train_y)
pred = model.predict(df_test_x_scaled)
accuracy_score(test_y, pred)

In [ ]:
df_pred = pd.DataFrame({
    'actual': test_y,
    'predicted': pred
})

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# tn, fp, fn, tp = confusion_matrix().ravel()

confusion_matrix(test_y, pred)

In [ ]:
tn, fp, fn, tp = confusion_matrix(test_y, pred).ravel()

In [ ]:
print(classification_report(test_y, pred))

In [ ]:
precision = 0.9
recall = 0.8
print('Sample mean:', np.mean([precision, recall]))
print('Harmonic mean:', 2 * (precision*recall) / (precision+recall))

In [ ]:
2 * (precision*recall) / (precision+recall)

# Decision Tree